In [10]:
import numpy as np
import pandas as pd
from glob import glob
import tensorflow as tf
import keras

In [11]:
# human antigens files
df_rep = pd.DataFrame(glob('../DeepTCR/Data/Human_Antigens/*/*.tsv'), columns=['filepath'])
df_rep[['HLA', 'Antigen']] = df_rep['filepath'].str.extract(r'/([^/-]+)-([^/-]+)/[^/]+$')

In [12]:
# load tcrseq data and label
df_tcr = []
for idx in df_rep.index:
    df_tcr.append(pd.read_csv(df_rep.loc[idx, 'filepath'], sep='\t'))
    df_tcr[-1]['index'] = idx
    df_tcr[-1][['HLA', 'Antigen']] = df_rep.loc[idx, ['HLA', 'Antigen']].values
df_tcr = pd.concat(df_tcr)
df_tcr['Antigen'] = df_tcr['Antigen'].astype('category')

In [13]:
# convert to tf dataset
idx = ~df_tcr[['aminoAcid', 'Antigen']].isna().any(axis=1)
X = df_tcr.loc[idx, 'aminoAcid'].values
y = df_tcr.loc[idx, 'Antigen'].cat.codes.values

In [14]:
y

array([], dtype=int8)

In [15]:
max_length = max(list(map(len, X)))
aa = set(''.join(X))
aa = dict(zip(aa, np.arange(1, len(aa) + 1)))

X = [[aa[l] for l in x] + [0] * (max_length - len(x)) for x in X]



ValueError: max() arg is an empty sequence

In [ ]:
tfds = tf.data.Dataset.from_tensor_slices((X, y))
tfds = tfds.map(lambda X, y_true: (X, tf.one_hot(y_true, len(np.unique(y)))))
tfds = tfds.shuffle(idx.sum(), reshuffle_each_iteration=True)
tfds = tfds.batch(100, drop_remainder=True)

2025-06-18 17:37:26.111609: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1048 MB memory:  -> device: 0, name: NVIDIA H100 80GB HBM3, pci bus id: 0000:5f:00.0, compute capability: 9.0


In [ ]:
tfds

<_BatchDataset element_spec=(TensorSpec(shape=(100, 23), dtype=tf.int32, name=None), TensorSpec(shape=(100, 7), dtype=tf.float32, name=None))>

In [ ]:
# CDR3 AA encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [ ]:
# VDJ usage encoding
tensors = []
tensors.append(keras.layers.Input(shape=(23, ),  dtype=tf.uint32))
tensors.append(keras.layers.Embedding(input_dim=len(aa) + 1, output_dim=64, mask_zero=True)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=64, kernel_size=9, strides=1, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=128, kernel_size=7, strides=2, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, activation=None)(tensors[-1])[:, 0, :])
encoder = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='encoder')

In [ ]:
tensors = []
tensors.append(keras.layers.Input(shape=(encoder.output_shape[-1], ), dtype=tf.float32))
tensors.append(keras.layers.Dropout(rate=0.1)(tensors[-1]))
tensors.append(keras.layers.Dense(units=128, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dropout(rate=0.05)(tensors[-1]))
tensors.append(keras.layers.Dense(units=64, activation=keras.activations.relu)(tensors[-1]))
tensors.append(keras.layers.Dense(units=len(np.unique(y)), activation=keras.activations.sigmoid)(tensors[-1]))
classifier = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='classifer')

In [ ]:
tensors = []
tensors.append(keras.layers.Input(shape=encoder.input_shape[1:],  dtype=tf.uint32))
tensors.append(encoder(tensors[-1]))
tensors.append(classifier(tensors[-1]))
model = keras.Model(inputs=tensors[0], outputs=tensors[-1], name='full_model')

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.BinaryCrossentropy(from_logits=False)
)

In [ ]:
model.fit(tfds, epochs=100)

Epoch 1/100


2025-06-18 17:37:27.455163: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-06-18 17:37:29.396878: I external/local_xla/xla/service/service.cc:168] XLA service 0x1550ca3b8f50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-18 17:37:29.396908: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA H100 80GB HBM3, Compute Capability 9.0
2025-06-18 17:37:29.401369: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1750282649.490841  614658 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 [==============================] - 5s 3ms/step - loss: 0.4751
Epoch 2/100
20/20 [==============================] - 0s 3ms/step - loss: 0.3584
Epoch 3/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3502
Epoch 4/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3512
Epoch 5/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3468
Epoch 6/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3426
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3295
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3155
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.3036
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2880
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.2691
Epoch 12/100
20/20 [==============================] - 0s 9ms/step - loss: 0.2496
Epoch 13/100
20/20 [=============================

In [ ]:
y_pred.shape

NameError: name 'y_pred' is not defined

In [ ]:
b = next(iter(tfds))

In [ ]:
b[0].shape, b[1].shape

(TensorShape([100, 23]), TensorShape([100, 7]))